In [3]:
import pandas as pd
import numpy as np
import operator
import random
import json
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

In [4]:
df = pd.read_csv('Diseases - Sheet1.csv')
df.head()

,Disease_Name,General_Symptom,Male_Symptoms,Female_Symptoms
0,Diabetes,"increased hunger, increased thirst, weight los...","a decreased sex drive, erectile dysfunction, b...","vaginal dryness, urinary tract infections, yea..."
1,Asthma,"coughing, chest pain, shortness of breath, dif...",NaN,NaN
2,Arthritis,"limited range of motion, clicking or popping w...",NaN,NaN
3,Malaria,"chills, fever, sweating, headache, nausea, vom...",NaN,NaN
4,Hepatitis,"fatigue, dark urine, pale stool, abdominal pai...",NaN,NaN


In [5]:
freq = {}
for i in df['General_Symptom']:
    s = i.split(', ')
    for j in s:
        if j not in freq:
            freq[j] = 1
        else:
            freq[j] += 1
sort = dict(sorted(freq.items(),key=operator.itemgetter(1),reverse=True))

In [6]:
disease = {}
for i in range(df.shape[0]):
    disease[df['Disease_Name'].values[i]] = df['General_Symptom'].values[i].split(", ")
disease

{'Diabetes': ['increased hunger',
  'increased thirst',
  'weight loss',
  'frequent urination',
  'blurry vision',
  'fatigue',
  'sores that don’t heal'],
 'Asthma': ['coughing',
  'chest pain',
  'shortness of breath',
  'difficulty talking',
  'anxiety',
  'fatigue',
  'chest pain',
  'rapid breathing',
  'frequent infections',
  'insomnia'],
 'Arthritis': ['limited range of motion',
  'clicking or popping with bending',
  'body ache',
  'instability of joint',
  'bony growths in the fingers',
  'grating or scraping feeling in the knees'],
 'Malaria': ['chills',
  'fever',
  'sweating',
  'headache',
  'nausea',
  'vomiting',
  'abdominal pain',
  'diarrhea',
  'body ache'],
 'Hepatitis': ['fatigue',
  'dark urine',
  'pale stool',
  'abdominal pain',
  'loss of appetite',
  'weight loss',
  'yellow skin and eyes'],
 'Pneumonia': ['coughing',
  'mucus',
  'fever',
  'chills',
  'shortness of breath',
  'chest pain',
  'fatigue',
  'nausea',
  'weight loss'],
 'Tuberculosis': ['fati

In [7]:
d1 = dict(list(sort.items())[:13])
common = list(d1.keys())

test = {}
count = 0

for i in disease.keys():
    for j in common:
        if j in disease[i]:
            count += 1
    test[i] = count / len(disease[i])
    count = 0
test

{'Diabetes': 0.42857142857142855,
 'Asthma': 0.5,
 'Arthritis': 0.16666666666666666,
 'Malaria': 0.5555555555555556,
 'Hepatitis': 0.42857142857142855,
 'Pneumonia': 0.8888888888888888,
 'Tuberculosis': 0.8333333333333334,
 'Sickle cell anemia': 0.2222222222222222,
 'Thalassemia': 0.14285714285714285,
 'Leukemia': 0.8571428571428571,
 'Lymphoma': 0.75,
 'Prostate cancer': 0.16666666666666666,
 'Colon cancer': 0.125}

In [8]:
for i in disease.keys():
    print(i)

Diabetes
Asthma
Arthritis
Malaria
Hepatitis
Pneumonia
Tuberculosis
Sickle cell anemia
Thalassemia
Leukemia
Lymphoma
Prostate cancer
Colon cancer


In [9]:
print("Pick your symptoms:")
# Search of other symptoms
active = []
final = []
for i in common:
    print(i)
print("0 to end")
inp = ""
for i in range(2):
    inp = input("You: ")
    active.append(inp)
done = False
flag = 0
while True:
    inp = input("You: ")
    if(inp == '-1'):
        flag += 1
    if(inp == '0'):
        break
    inp += ", "
    for i in inp.split(", "):
        active.append(i)
    active.pop(-1)
    # print(active)
    prob = {}
    for key, value in disease.items():
        prob[key] = 0
        for i in active:
            if i in value:
                prob[key] += 1
        prob[key] = prob[key] / len(value)
    sorted_prob = dict(sorted(prob.items(),key=operator.itemgetter(1),reverse=True))
    print(sorted_prob)
    display = []
    for key, value in sorted_prob.items():
        if value >= 0.8:
            final.append(key)
            done = True
        else:
            break
    if(done == True):
        break
    for key in list(sorted_prob.keys())[(flag * 3):]:
        count = 0
        if(len(display) >= 10):
            break
        for i in disease[key]:
            if count == 3:
                continue
            if i not in active and i not in display:
                display.append(i)
                count += 1
    print("Choose - ")
    for i in display:
        print(i)
    print("If Exit - 0\nIf None - -1")
print(final)

Pick your symptoms:
fatigue
weight loss
fever
shortness of breath
chills
coughing
chest pain
frequent infections
abdominal pain
sweats
frequent urination
body ache
nausea
0 to end
{'Tuberculosis': 0.16666666666666666, 'Diabetes': 0.14285714285714285, 'Hepatitis': 0.14285714285714285, 'Thalassemia': 0.14285714285714285, 'Leukemia': 0.14285714285714285, 'Lymphoma': 0.125, 'Pneumonia': 0.1111111111111111, 'Sickle cell anemia': 0.1111111111111111, 'Asthma': 0.1, 'Arthritis': 0.0, 'Malaria': 0.0, 'Prostate cancer': 0.0, 'Colon cancer': 0.0}
Choose - 
fever
chills
sweats
increased hunger
increased thirst
weight loss
dark urine
pale stool
abdominal pain
bone deformities in face
delayed growth and development
yellow or pale skin
If Exit - 0
If None - -1
{'Hepatitis': 0.2857142857142857, 'Thalassemia': 0.2857142857142857, 'Tuberculosis': 0.16666666666666666, 'Diabetes': 0.14285714285714285, 'Leukemia': 0.14285714285714285, 'Lymphoma': 0.125, 'Pneumonia': 0.1111111111111111, 'Sickle cell anemia'

In [12]:
lemmatizer = WordNetLemmatizer()
final = 'Diabetes'
intents = json.loads(open(f'Diseases/Diabetes/intents.json').read())

words = pickle.load(open(r'Diseases/Diabetes/words.pkl', 'rb'))
classes = pickle.load(open(r'Diseases/Diabetes/classes.pkl', 'rb'))
model = load_model(f'Diseases/Diabetes/diabetes.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word)  for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words= clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda  x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list


def get_response(intents_list,intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

while True:
    message = input("You: ")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print("Doctor:", res)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/tejasmadhukar/nltk_data'
    - '/opt/homebrew/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/nltk_data'
    - '/opt/homebrew/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/share/nltk_data'
    - '/opt/homebrew/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [13]:
prob = {}
for key, value in disease.items():
    prob[key] = 0
    for i in active:
        if i in value:
            prob[key] += 1
    prob[key] = prob[key] / len(value)
prob

{'Diabetes': 0.42857142857142855,
 'Asthma': 0.1,
 'Arthritis': 0.0,
 'Malaria': 0.0,
 'Hepatitis': 0.42857142857142855,
 'Pneumonia': 0.2222222222222222,
 'Tuberculosis': 0.3333333333333333,
 'Sickle cell anemia': 0.1111111111111111,
 'Thalassemia': 0.42857142857142855,
 'Leukemia': 0.2857142857142857,
 'Lymphoma': 0.125,
 'Prostate cancer': 0.16666666666666666,
 'Colon cancer': 0.0}